In [11]:
import os

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

from utils.pd import deduplicated

# 加载 .env 文件
load_dotenv("d:/.env")

host = os.getenv("UNI_CLI_MYSQL_HOST")
user = os.getenv("UNI_CLI_MYSQL_USER")
password = os.getenv("UNI_CLI_MYSQL_PASSWORD")
database = os.getenv("UNI_CLI_MYSQL_DATABASE")

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")


# 查询数据
query = "select id,created_at,name,act_name,symbol,qty,cex,status,up_status,path,level,earn,cost,buy_px,benefit,sell_px,profit,order_id,client_order_id,fx_order_id,fx_client_order_id,open_at,close_at from tx where profit is not null and up_status in (0,1);"
data_frame = pd.read_sql(query, engine)
# 提取 'id' 列
ids = data_frame["id"].tolist()
# 删除 'id' 列
data_frame = data_frame.drop(columns=["id"])
# 将数据追加写入 CSV 文件
csv_path = "d:/github/txnj/data/bitget_grid_0.csv"
data_frame.to_csv(
    csv_path,
    mode="a",
    header=not os.path.exists(csv_path),
    index=False,
    encoding="utf-8",
)
# csv去重,保留最后加入的数据
deduplicated(csv_path, "client_order_id", "last")

# 根据提取的 'id' 列更新数据库中 up_status 字段为 2
if ids:
    update_query = text(
        f"UPDATE tx SET up_status = 2 WHERE id IN ({','.join(map(str, ids))});"
    )
    with engine.connect() as connection:
        with connection.begin():
            connection.execute(update_query)
data_frame

,created_at,name,act_name,symbol,qty,cex,status,up_status,path,level,...,buy_px,benefit,sell_px,profit,order_id,client_order_id,fx_order_id,fx_client_order_id,open_at,close_at
